#Importaciones

In [1]:
!pip install pulp

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python312\\Scripts\\pulptest.exe' -> 'C:\\Python312\\Scripts\\pulptest.exe.deleteme'


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB 640.0 kB/s eta 0:00:28
   ---------------------------------------- 0.0/17.7 MB 487.6 kB/s eta 0:00:37
   ---------------------------------------- 0.2/17.7 MB 1.5 MB/s eta 0:00:12
   - -------------------------------------- 0.5/17.7 MB 3.0 MB/s eta 0:00:06
   -- ------------------------------------- 1.0/17.7 MB 4.8 MB/s eta 0:00:04
   --- ------------------------------------ 1.6/17.7 MB 6.3 MB/s eta 0:00:03
   ----- ---------------------------------- 2.6/17.7 MB 8.7 MB/s eta 0:00:02
   ---------- ----------------------------- 4.7/17.7 MB 13.6 MB/s eta 0:00:01
   -------------- ------------------------- 6.5/17.7 MB 16.7 MB/s eta 0:00:01
   ------------------- -------------------- 8.8/17.7 MB 20.1 MB/s eta 0:00:01
   ------------------------ --------------- 10.9/17.7 MB 34.4 MB/s eta 0:00:01
   ---------------------------- ----------- 12.8/17.7 MB 43.7 MB/s eta 0:

In [1]:
import random
import json
import pulp
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpBinary
from datetime import datetime, timedelta

#Partidos


In [2]:
class Partido:
    def __init__(self, numero, equipo_local, equipo_visitante, fecha_hora, cancha, arbitro):
        self.numero = numero
        self.equipo_local = equipo_local
        self.equipo_visitante = equipo_visitante
        self.fecha_hora = fecha_hora
        self.cancha = cancha
        self.arbitro = arbitro

class Calendario:
    def __init__(self):
        self.partidos = []
        self.canchas = set()
        self.arbitros = set()
        self.contador_partidos = 1

    def agregar_partido(self, partido):
        partido.numero = self.contador_partidos
        self.partidos.append(partido)
        self.contador_partidos += 1
        self.canchas.add(partido.cancha)
        self.arbitros.add(partido.arbitro)

    def generar_calendario_optimo(self, fecha_inicio_torneo, dias_semana, hora_inicio, hora_fin):
        # Convertir días de la semana a números (0 es lunes y 6 es domingo)
        dias_semana_numericos = [datetime.strptime(dia, "%A").weekday() for dia in dias_semana]

        # Crear problema de optimización
        problema = LpProblem("Calendario de Partidos", LpMinimize)

        # Variables de decisión
        X = {(partido, cancha, hora, dia): LpVariable(f'X_{partido.numero}_{cancha}_{hora}_{dia}', cat=LpBinary)
             for partido in self.partidos
             for cancha in self.canchas
             for hora in range(hora_inicio, hora_fin)  # Rango de horas
             for dia in range(len(dias_semana_numericos))}  # Considerar todas las semanas

        # Minimizar el número de conflictos
        problema += lpSum(X[partido, cancha, hora, dia] for partido in self.partidos
                          for cancha in self.canchas
                          for hora in range(hora_inicio, hora_fin)  # Rango de horas
                          for dia in range(len(dias_semana_numericos)))

        # Restricciones
        for partido in self.partidos:
            for cancha in self.canchas:
                for hora in range(hora_inicio, hora_fin):
                    for dia in range(len(dias_semana_numericos)):
                        # Cada partido se juega en exactamente una cancha, hora y día
                        problema += lpSum(X[p, c, h, d] for p, c, h, d in X if p == partido) == 1
                        # No pueden haber dos partidos en la misma cancha al mismo tiempo y día
                        problema += lpSum(X[p, c, h, d] for p, c, h, d in X if c == cancha and h == hora and d == dia) <= 1

        # Resolver el problema
        problema.solve()

        for partido, cancha, hora, dia in X:
            if X[partido, cancha, hora, dia].value() == 1:
                fecha_partido = datetime.strptime(fecha_inicio_torneo, "%Y-%m-%d") + timedelta(weeks=dia, days=dias_semana_numericos[dia])
                fecha_hora = datetime(fecha_partido.year, fecha_partido.month, fecha_partido.day, hora)
                print(f"Partido {partido.numero}: {partido.equipo_local} vs {partido.equipo_visitante}, en la cancha {cancha} a las {fecha_hora.strftime('%H:%M')} el día {fecha_partido.strftime('%Y-%m-%d')}")


#manual

In [3]:
# Ejemplo de uso
calendario = Calendario()

equipos = [f"Equipo {i}" for i in range(1, 51)]
canchas = [f"Cancha {i}" for i in range(1, 21)]
arbitros = [f"Árbitro {i}" for i in range(1, 16)]

# Agregar 50 partidos con equipos aleatorios, canchas y árbitros
for i in range(50):
    fecha_hora = datetime.strptime("2024-03-22", "%Y-%m-%d") + timedelta(days=i)
    partido = Partido(None, equipos[i], equipos[(i+1) % 50], fecha_hora, canchas[i % 20], arbitros[i % 15])
    calendario.agregar_partido(partido)

print("Calendario de partidos:")
# Solicitar fecha de inicio del torneo
fecha_inicio_torneo = input("Ingrese la fecha de inicio del torneo (YYYY-MM-DD): ")
# Solicitar días de la semana en los que se jugarán los partidos
dias_semana = input("Ingrese los días de la semana en los que se jugarán los partidos (separados por comas, en inglés): ").split(", ")
# Solicitar hora de inicio y fin del rango de horas en la que se jugarán los partidos
hora_inicio = int(input("Ingrese la hora de inicio del rango de horas (0-23): "))
hora_fin = int(input("Ingrese la hora de fin del rango de horas (1-24): "))

calendario.generar_calendario_optimo(fecha_inicio_torneo, dias_semana, hora_inicio, hora_fin)

Calendario de partidos:


ValueError: invalid literal for int() with base 10: ''

#Con JSON


In [ ]:
def obtener_datos(file_name):
        with open(file_name, 'r') as file:
            data = json.load(file)
        return data

In [ ]:
datos = obtener_datos('test.json')

In [ ]:
class Partido:
    def __init__(self, numero, equipo_local, equipo_visitante=None, fecha_hora=None, cancha=None, arbitro=None):
        self.numero = numero
        self.equipo_local = equipo_local
        self.equipo_visitante = equipo_visitante
        self.fecha_hora = fecha_hora
        self.cancha = cancha
        self.arbitro = arbitro

class Calendario:
    def __init__(self):
        self.partidos = []
        self.canchas = set()
        self.arbitros = set()
        self.contador_partidos = 1

    def agregar_partido(self, partido):
        partido.numero = self.contador_partidos
        self.partidos.append(partido)
        self.contador_partidos += 1
        if partido.cancha:
            self.canchas.add(partido.cancha)
        if partido.arbitro:
            self.arbitros.add(partido.arbitro)

    def generar_calendario_optimo(self, fecha_inicio_torneo, dias_semana, hora_inicio, hora_fin, output_file):
        # Convertir días de la semana a números (0 es lunes y 6 es domingo)
        dias_semana_numericos = [datetime.strptime(dia, "%A").weekday() for dia in dias_semana]

        # Reorganizar aleatoriamente la lista de equipos
        random.shuffle(equipos)

        # Crear problema de optimización
        problema = LpProblem("Calendario de Partidos", LpMinimize)

        # Variables de decisión
        X = {(partido, cancha, hora, dia): LpVariable(f'X_{partido.numero}_{cancha}_{hora}_{dia}', cat=LpBinary)
             for partido in self.partidos
             for cancha in self.canchas
             for hora in range(hora_inicio, hora_fin)  # Rango de horas
             for dia in range(len(dias_semana_numericos))}  # Considerar todas las semanas

        # Minimizar el número de conflictos
        problema += lpSum(X[partido, cancha, hora, dia] for partido in self.partidos
                          for cancha in self.canchas
                          for hora in range(hora_inicio, hora_fin)  # Rango de horas
                          for dia in range(len(dias_semana_numericos)))

        # Restricciones
        for partido in self.partidos:
            for cancha in self.canchas:
                for hora in range(hora_inicio, hora_fin):
                    for dia in range(len(dias_semana_numericos)):
                        # Cada partido se juega en exactamente una cancha, hora y día
                        problema += lpSum(X[p, c, h, d] for p, c, h, d in X if p == partido) == 1
                        # No pueden haber dos partidos en la misma cancha al mismo tiempo y día
                        problema += lpSum(X[p, c, h, d] for p, c, h, d in X if c == cancha and h == hora and d == dia) <= 1

        # Resolver el problema
        problema.solve()

        partidos_json = []
        for partido, cancha, hora, dia in X:
            if X[partido, cancha, hora, dia].value() == 1:
                fecha_partido = datetime.strptime(fecha_inicio_torneo, "%Y-%m-%d") + timedelta(weeks=dia, days=dias_semana_numericos[dia])
                fecha_hora = datetime(fecha_partido.year, fecha_partido.month, fecha_partido.day, hora)
                partido_info = {
                    "numero": partido.numero,
                    "equipo_local": partido.equipo_local,
                    "equipo_visitante": partido.equipo_visitante if partido.equipo_visitante else "Bye",
                    "fecha_hora": fecha_hora.strftime('%Y-%m-%d %H:%M'),
                    "cancha": cancha,
                    "arbitro": partido.arbitro if partido.arbitro else "No asignado"
                }
                partidos_json.append(partido_info)

        with open(output_file, 'w') as json_file:
            json.dump(partidos_json, json_file, indent=4)


In [4]:
from datetime import date

today = date.today()
print("Today's date is:", today)

Today's date is: 2024-04-24


In [ ]:
# Ejemplo de uso
calendario = Calendario()

equipos = datos["Equipos"]
canchas = datos["Canchas"]
arbitros = datos["Arbitros"]
dias_semana_origen = datos["DiasSemana"]

for i in range(0, len(equipos), 2):
    if i + 1 < len(equipos):
        # Generar una fecha y hora únicas para cada partido
        fecha_hora = datetime.strptime("2024-03-22", "%Y-%m-%d") + timedelta(days=i * (len(equipos) // 2) + i // 2)
        partido = Partido(None, equipos[i], equipos[i + 1], fecha_hora, canchas[i % len(canchas)], arbitros[i % len(arbitros)])
        calendario.agregar_partido(partido)
    else:
        # Para el último equipo, crea un bye
        partido = Partido(None, "Equipo sin rival: ", equipos[i])
        calendario.agregar_partido(partido)

# Solicitar fecha de inicio del torneo
fecha_inicio_torneo = date.today()#input("Ingrese la fecha de inicio del torneo (YYYY-MM-DD): ")
# Solicitar días de la semana en los que se jugarán los partidos
dias_semana = dias_semana_origen#input("Ingrese los días de la semana en los que se jugarán los partidos (separados por comas, en inglés): ").split(", ")
# Solicitar hora de inicio y fin del rango de horas en la que se jugarán los partidos
hora_inicio = 6#int(input("Ingrese la hora de inicio del rango de horas (0-23): "))
hora_fin = 18#int(input("Ingrese la hora de fin del rango de horas (1-24): "))
# Solicitar nombre del archivo de salida
output_file = "Torneo" + date.today() + ".json"

calendario.generar_calendario_optimo(fecha_inicio_torneo, dias_semana, hora_inicio, hora_fin, output_file)

Ingrese la fecha de inicio del torneo (YYYY-MM-DD): 2024-03-15
Ingrese los días de la semana en los que se jugarán los partidos (separados por comas, en inglés): Friday, Saturday, Sunday
Ingrese la hora de inicio del rango de horas (0-23): 10
Ingrese la hora de fin del rango de horas (1-24): 17
Ingrese el nombre del archivo de salida (sin extensión): Calendario
